<a href="https://www.kaggle.com/code/avikumart/healthcare-datasci-retinal-fundus-segmentation?scriptVersionId=265398220" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import numpy as np
from PIL import Image
import cv2
from scipy.ndimage import binary_opening
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization, ReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2025-09-30 15:59:41.891906: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759247982.147696      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759247982.221686      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/training/images/34_training.tif
/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/training/images/40_training.tif
/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/training/images/22_training.tif
/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/training/images/27_training.tif
/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/training/images/28_training.tif
/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/training/images/26_training.tif
/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/training/images/38_training.tif
/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/training/images/39_training.tif
/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/training/images/24_training.tif
/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/tr

In [8]:
# datasets path

# Dataset paths
train_images_path = "/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/training/images"
train_manual_path = "/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/training/1st_manual"
train_mask_path = "/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/training/mask"
test_images_path = "/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/test/images"
test_mask_path = "/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/test/mask"

def check_dir(path):
    if os.path.exists(path):
        print(f"{path} exists")
    else:
        print(f"{path} does not exist")
    return os.path.exists(path)

# check the dirs
check_dir(train_images_path)
check_dir(test_images_path)

/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/training/images exists
/kaggle/input/drive-digital-retinal-images-for-vessel-extraction/DRIVE/test/images exists


True

In [1]:
# load the images
def load_images(img_path):
    img = Image.open(img_path)
    if img.mod == "RGB":
        img = np.array(img)
        green_channel = img[:,:,1]
        return green_channel
    else:
        return np.array(img)

In [ ]:
def preprocess_image(image):
    